The Problem definition
Data = cars.csv(Uploaded in github data part)
Predictors = [Cylinders, Weight, Horsepower]
Target = MPG

You will remove all the observations for which MPG and Horsepower is zero.
Do a scaling - do standardization of all the predictors ((X-mean)/sd) [This won't make it standard normal distributed) - This is done beacuse different variables are measured in different scales.

Keep aside one random observation from the dataset as test. (DO NOT include that observation for training the algorithm)

you need to build a kNN algorithm for regression.

kNN(trainX, trainY, testX, k)

In [115]:
import os
os.getcwd()
import pandas as pd
from sklearn.preprocessing import StandardScaler 

In [116]:
# Set the working directory and read the CSV file
os.chdir('/Users/vinayashreek/Desktop/DataScience/Praxis/Coding/Praxis/LearningIML/DataSetBySir')
cars = pd.read_csv("cars.csv")

In [117]:
#Updating selected features and label to the car updated list
cars_updated = cars[["Cylinders","Weight","Horsepower","MPG"]]

In [118]:
#Checking if minimum value of MPG or Horsepower is zero
print("MPG : ",cars.MPG.min())
print("Horsepower : ",cars.Horsepower.min())

MPG :  0.0
Horsepower :  0


In [119]:
#We find that minimum value of both are zero.Hence we need to remove the rows containing zero in those 2 columns
#Lets take only rows where MPG or Horsepower is not zero
cars_updated = cars_updated[(cars_updated[['MPG','Horsepower']] != 0).all(axis=1)]

In [120]:
#Lets recheck if minimum value of MPG or Horsepower is still zero
print("MPG : ",cars_updated.MPG.min())
print("Horsepower : ",cars_updated.Horsepower.min())

MPG :  9.0
Horsepower :  46


In [121]:
#We can confirm that rows with zeros are removed now since min values of both are not zero now
#calculating the table length after removng zeros
table_length = len(cars_updated)
table_length

392

In [122]:
cars_updated.head()

,Cylinders,Weight,Horsepower,MPG
0,8,3504,130,18.0
1,8,3693,165,15.0
2,8,3436,150,18.0
3,8,3433,150,16.0
4,8,3449,140,17.0


In [123]:
#Choosing the train set.Here I am taking from second row till the end 
#Features I am using are given as a second parameter in loc
trainX = cars_updated.loc[1:392,["Cylinders","Weight","Horsepower"]]
trainX.head()

,Cylinders,Weight,Horsepower
1,8,3693,165
2,8,3436,150
3,8,3433,150
4,8,3449,140
5,8,4341,198


In [124]:
#This is the label of the training set.Label for all the training set rows are givem
trainY = cars_updated.MPG[1:392]
trainY.head()

1    15.0
2    18.0
3    16.0
4    17.0
5    15.0
Name: MPG, dtype: float64

In [125]:
#Choosing the test set.Here I am taking only the first row as my test set 
#Features I am using are given as a second parameter in loc
test = cars_updated.loc[0,["Cylinders","Weight","Horsepower"]]
test.head()

Cylinders        8.0
Weight        3504.0
Horsepower     130.0
Name: 0, dtype: float64

In [126]:
#I used following library to standardize all the predictors of car_updated and put it in a new copy of the data frame called S_cars_updated
from sklearn.preprocessing import StandardScaler
S_cars_updated = cars_updated.copy()
S_cars_updated[['Cylinders', 'Weight','Horsepower','MPG']] = StandardScaler().fit_transform(S_cars_updated[['Cylinders', 'Weight','Horsepower','MPG']])
S_cars_updated.head()

,Cylinders,Weight,Horsepower,MPG
0,1.483947,0.620540,0.664133,-0.698638
1,1.483947,0.843334,1.574594,-1.083498
2,1.483947,0.540382,1.184397,-0.698638
3,1.483947,0.536845,1.184397,-0.955212
4,1.483947,0.555706,0.924265,-0.826925


In [141]:
#Assigning the standardised values of training and test set to the follwing names
S_trainX = S_cars_updated.loc[1:392,["Cylinders","Weight","Horsepower"]]
S_test = S_cars_updated.loc[0,["Cylinders","Weight","Horsepower"]]
print(S_trainX.head())
print(S_test.head)

   Cylinders    Weight  Horsepower
1   1.483947  0.843334    1.574594
2   1.483947  0.540382    1.184397
3   1.483947  0.536845    1.184397
4   1.483947  0.555706    0.924265
5   1.483947  1.607198    2.433030
<bound method NDFrame.head of Cylinders     1.483947
Weight        0.620540
Horsepower    0.664133
Name: 0, dtype: float64>


In [128]:
#Just confirming if standardization has been applied by taking first value of cylinders and comparing it with hand calculated value
M=cars.Cylinders.mean()
S=cars.Cylinders.std()
print("standardized value from above table using code",S_cars_updated.Cylinders[0])
print("standardized value via calculation",(cars.Cylinders[0]-M)/S)


standardized value from above table using code 1.483947024779763
standardized value via calculation 1.4745298834014493


In [129]:
#Used the following libraries to calculate the euclidean distance
from scipy.spatial import distance
import numpy as np
#Function for calculating the knn given the features in train set as train_x
#label in train set as train y,test set as test (Passing the standardised predictors)
def knn(train_x,train_y, test, k):
    dist = []
    for i in range(0, len(train_x)):
        dist.append(distance.euclidean(test, train_x.iloc[i,[0,1,2]]))

    dist_copy = dist.copy()
    dist.sort()
    index = []
    for i in range(0,k):
        indx = dist_copy.index(dist[i])
        index.append(indx)
        dist_copy[indx] = -1
    pred_y = np.mean(train_y.iloc[index])
    return(pred_y)

In [136]:
#Finally finding the label for the test set using the train_x and train_y
#Different values of K gives the value with different accuracy.
#I have used k = 6
knn(S_trainX,trainY,S_test,k=6)

17.733333333333334

<b>The expected value was 18
Our model with k=6 predicted 17.7333333
Thats pretty good:)</b>